# Galactic Dynamics Package Examples

This notebook demonstrates the usage of the galactic dynamics package we've developed. We'll explore various components of the package and visualize the results.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from galactic_dynamics import (
    mass_density, enclosed_mass, analytical_mass,
    gas_density, pressure_gradient, turbulence_pressure,
    magnetic_field_strength, magnetic_pressure,
    cosmic_ray_pressure, solve_velocity
)
from galactic_dynamics.constants import pc, M_sun, G, k_B, m_p

%matplotlib inline
plt.style.use('seaborn')

## 1. Mass Models

Let's start by exploring the mass distribution in our galaxy model.

In [ ]:
# Define galaxy parameters
galaxy_params = (3e3*pc, 5e10*M_sun, 500*pc, 1e10*M_sun, 20e3*pc, 1e12*M_sun)
r_d, M_d, r_b, M_b, r_h, M_h = galaxy_params

# Create radial points
r = np.logspace(np.log10(10*pc), np.log10(100e3*pc), 1000)

# Calculate mass density and enclosed mass
density = mass_density(r, galaxy_params)
mass = enclosed_mass(r, galaxy_params)
mass_analytic = analytical_mass(r, galaxy_params)

# Plot results
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

ax1.loglog(r/pc, density)
ax1.set_xlabel('Radius (pc)')
ax1.set_ylabel('Density (kg/m³)')
ax1.set_title('Mass Density Profile')

ax2.loglog(r/pc, mass/M_sun, label='Numerical')
ax2.loglog(r/pc, mass_analytic/M_sun, '--', label='Analytical')
ax2.set_xlabel('Radius (pc)')
ax2.set_ylabel('Enclosed Mass (M_sun)')
ax2.set_title('Enclosed Mass')
ax2.legend()

plt.tight_layout()
plt.show()

## 2. Gas Models

Now let's examine the gas distribution and its properties.

In [ ]:
# Define gas parameters
gas_params = (4e3*pc, 1e10*M_sun, 1e4, 1e4)
r_g, M_g, T, v_turb = gas_params

# Calculate gas properties
rho_gas = gas_density(r, r_g, M_g)
p_grad = pressure_gradient(r, r_g, M_g, T)
p_turb = turbulence_pressure(r, r_g, M_g, v_turb)

# Plot results
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 6))

ax1.loglog(r/pc, rho_gas)
ax1.set_xlabel('Radius (pc)')
ax1.set_ylabel('Gas Density (kg/m³)')
ax1.set_title('Gas Density Profile')

ax2.semilogx(r/pc, p_grad)
ax2.set_xlabel('Radius (pc)')
ax2.set_ylabel('Pressure Gradient (Pa/m)')
ax2.set_title('Gas Pressure Gradient')

ax3.loglog(r/pc, p_turb)
ax3.set_xlabel('Radius (pc)')
ax3.set_ylabel('Turbulence Pressure (Pa)')
ax3.set_title('Turbulence Pressure')

plt.tight_layout()
plt.show()

## 3. Magnetic Fields and Cosmic Rays

Let's explore the effects of magnetic fields and cosmic rays in our galaxy model.

In [ ]:
# Define magnetic and cosmic ray parameters
B0 = 1e-10  # Tesla
r_B = 5e3*pc
P_cr0 = 1e-12  # Pa
r_cr = 10e3*pc

# Calculate magnetic and cosmic ray properties
B = magnetic_field_strength(r, B0, r_B)
p_mag = magnetic_pressure(B)
p_cr = cosmic_ray_pressure(r, P_cr0, r_cr)

# Plot results
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 6))

ax1.loglog(r/pc, B)
ax1.set_xlabel('Radius (pc)')
ax1.set_ylabel('Magnetic Field Strength (T)')
ax1.set_title('Magnetic Field Profile')

ax2.loglog(r/pc, p_mag)
ax2.set_xlabel('Radius (pc)')
ax2.set_ylabel('Magnetic Pressure (Pa)')
ax2.set_title('Magnetic Pressure')

ax3.loglog(r/pc, p_cr)
ax3.set_xlabel('Radius (pc)')
ax3.set_ylabel('Cosmic Ray Pressure (Pa)')
ax3.set_title('Cosmic Ray Pressure')

plt.tight_layout()
plt.show()

## 4. Rotation Curves

Finally, let's calculate and plot the rotation curves for our galaxy model.

In [ ]:
# Combine all parameters
full_gas_params = (r_g, M_g, T, v_turb, B0, r_B, 0.1, P_cr0, r_cr, -1e3)

# Calculate rotation curves
v_odeint = solve_velocity(r, galaxy_params, full_gas_params, method='odeint')
v_rk4 = solve_velocity(r, galaxy_params, full_gas_params, method='rk4')
v_keplerian = np.sqrt(G * enclosed_mass(r, galaxy_params) / r)

# Plot results
plt.figure(figsize=(12, 8))
plt.semilogx(r/pc, v_odeint/1000, label='ODEINT')
plt.semilogx(r/pc, v_rk4/1000, '--', label='RK4')
plt.semilogx(r/pc, v_keplerian/1000, ':', label='Keplerian')
plt.xlabel('Radius (pc)')
plt.ylabel('Rotation Velocity (km/s)')
plt.title('Galaxy Rotation Curve')
plt.legend()
plt.grid(True)
plt.show()

# Calculate and print the average differences
avg_diff_odeint = np.mean((v_odeint - v_keplerian) / v_keplerian) * 100
avg_diff_rk4 = np.mean((v_rk4 - v_keplerian) / v_keplerian) * 100
print(f"ODEINT: Average difference from Keplerian: {avg_diff_odeint:.2f}%")
print(f"RK4: Average difference from Keplerian: {avg_diff_rk4:.2f}%")
print(f"Difference between ODEINT and RK4: {np.mean(np.abs(v_odeint - v_rk4))/1000:.4f} km/s")

## 5. Parameter Exploration

Let's explore how changing various parameters affects the rotation curve.

In [ ]:
def plot_rotation_curve(galaxy_params, gas_params, label):
    v = solve_velocity(r, galaxy_params, gas_params, method='rk4')
    plt.semilogx(r/pc, v/1000, label=label)

# Base parameters
base_galaxy_params = (3e3*pc, 5e10*M_sun, 500*pc, 1e10*M_sun, 20e3*pc, 1e12*M_sun)
base_gas_params = (4e3*pc, 1e10*M_sun, 1e4, 1e4, 1e-10, 5e3*pc, 0.1, 1e-12, 10e3*pc, -1e3)

plt.figure(figsize=(12, 8))

# Base case
plot_rotation_curve(base_galaxy_params, base_gas_params, 'Base Case')

# Increase halo mass
high_halo_params = base_galaxy_params[:4] + (base_galaxy_params[4], 2e12*M_sun)
plot_rotation_curve(high_halo_params, base_gas_params, 'High Halo Mass')

# Increase gas mass
high_gas_params = (base_gas_params[0], 5e10*M_sun) + base_gas_params[2:]
plot_rotation_curve(base_galaxy_params, high_gas_params, 'High Gas Mass')

# Increase magnetic field strength
high_b_params = base_gas_params[:4] + (1e-9,) + base_gas_params[5:]
plot_rotation_curve(base_galaxy_params, high_b_params, 'Strong Magnetic Field')

plt.xlabel('Radius (pc)')
plt.ylabel('Rotation Velocity (km/s)')
plt.title('Galaxy Rotation Curves - Parameter Exploration')
plt.legend()
plt.grid(True)
plt.show()

## 6. Comparison with Observational Data

Let's compare our model with some mock observational data.

In [ ]:
# Generate mock observational data
r_obs = np.logspace(np.log10(1e3*pc), np.log10(50e3*pc), 20)
v_obs = solve_velocity(r_obs, base_galaxy_params, base_gas_params, method='rk4')
v_obs += np.random.normal(0, 5e3, v_obs.shape)  # Add some noise

# Calculate model prediction
v_model = solve_velocity(r, base_galaxy_params, base_gas_params, method='rk4')

plt.figure(figsize=(12, 8))
plt.semilogx(r/pc, v_model/1000, label='Model')
plt.errorbar(r_obs/pc, v_obs/1000, yerr=5, fmt='o', label='Mock Observations')
plt.xlabel('Radius (pc)')
plt.ylabel('Rotation Velocity (km/s)')
plt.title('Galaxy Rotation Curve: Model vs. Mock Observations')
plt.legend()
plt.grid(True)
plt.show()

# Calculate residuals
v_model_interp = np.interp(r_obs, r, v_model)
residuals = (v_obs - v_model_interp) / 1000  # in km/s

plt.figure(figsize=(12, 6))
plt.semilogx(r_obs/pc, residuals, 'o')
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Radius (pc)')
plt.ylabel('Residual Velocity (km/s)')
plt.title('Residuals: Observations - Model')
plt.grid(True)
plt.show()

print(f"RMS of residuals: {np.sqrt(np.mean(residuals**2)):.2f} km/s")

## 7. Dark Matter Halo Profile

Let's examine the dark matter halo profile in our model and compare it to common profiles like NFW and Einasto.

In [ ]:
def nfw_profile(r, rho_s, r_s):
    return rho_s / ((r/r_s) * (1 + r/r_s)**2)

def einasto_profile(r, rho_e, r_e, n):
    return rho_e * np.exp(-2 * ((r/r_e)**(1/n) - 1) / n)

# Extract halo parameters
r_h, M_h = base_galaxy_params[4], base_galaxy_params[5]

# Calculate halo density
rho_halo = M_h / (4 * np.pi * r_h**3) * (1 + r/r_h)**-2

# Fit NFW profile
from scipy.optimize import curve_fit
popt_nfw, _ = curve_fit(nfw_profile, r, rho_halo)
rho_s, r_s = popt_nfw

# Fit Einasto profile
def einasto_fit(r, rho_e, r_e):
    return einasto_profile(r, rho_e, r_e, n=6)  # Fixing n=6 for simplicity

popt_einasto, _ = curve_fit(einasto_fit, r, rho_halo)
rho_e, r_e = popt_einasto

plt.figure(figsize=(12, 8))
plt.loglog(r/pc, rho_halo, label='Model Halo')
plt.loglog(r/pc, nfw_profile(r, rho_s, r_s), '--', label='NFW Fit')
plt.loglog(r/pc, einasto_profile(r, rho_e, r_e, 6), ':', label='Einasto Fit')
plt.xlabel('Radius (pc)')
plt.ylabel('Density (kg/m³)')
plt.title('Dark Matter Halo Density Profile')
plt.legend()
plt.grid(True)
plt.show()

print(f"NFW fit parameters: rho_s = {rho_s:.2e} kg/m³, r_s = {r_s/pc:.2f} pc")
print(f"Einasto fit parameters: rho_e = {rho_e:.2e} kg/m³, r_e = {r_e/pc:.2f} pc")

## 8. Mass-to-Light Ratio

Let's calculate and plot the mass-to-light ratio as a function of radius, assuming a simple stellar population model.

In [ ]:
def stellar_mass_to_light(r, galaxy_params):
    r_d, M_d, r_b, M_b, _, _ = galaxy_params
    stellar_mass = M_d * (1 - (1 + r/r_d) * np.exp(-r/r_d)) + M_b * r**2 / (r + r_b)**2
    # Assume a simple M/L ratio for the stellar component
    return stellar_mass / (3 * stellar_mass / M_sun)  # Assuming 3 L_sun per M_sun

def total_mass_to_light(r, galaxy_params):
    total_mass = enclosed_mass(r, galaxy_params)
    stellar_light = stellar_mass_to_light(r, galaxy_params) * (3 * M_sun)  # Convert back to mass then to light
    return total_mass / stellar_light

M_L_stellar = stellar_mass_to_light(r, base_galaxy_params)
M_L_total = total_mass_to_light(r, base_galaxy_params)

plt.figure(figsize=(12, 8))
plt.semilogx(r/pc, M_L_stellar, label='Stellar M/L')
plt.semilogx(r/pc, M_L_total, label='Total M/L')
plt.xlabel('Radius (pc)')
plt.ylabel('Mass-to-Light Ratio (M_sun/L_sun)')
plt.title('Mass-to-Light Ratio vs. Radius')
plt.legend()
plt.grid(True)
plt.show()

## 9. Tully-Fisher Relation

Let's explore how our model compares to the Tully-Fisher relation by varying the total mass of the galaxy.

In [ ]:
def scale_galaxy(base_params, scale_factor):
    return tuple(p * scale_factor if i % 2 == 1 else p for i, p in enumerate(base_params))

mass_scales = np.logspace(-1, 1, 20)
v_max = []

for scale in mass_scales:
    scaled_params = scale_galaxy(base_galaxy_params, scale)
    v = solve_velocity(r, scaled_params, base_gas_params, method='rk4')
    v_max.append(np.max(v))

total_masses = np.array([sum(scale_galaxy(base_galaxy_params, s)[1::2]) for s in mass_scales])

plt.figure(figsize=(12, 8))
plt.loglog(total_masses/M_sun, np.array(v_max)/1000, 'o-')
plt.xlabel('Total Mass (M_sun)')
plt.ylabel('Maximum Rotation Velocity (km/s)')
plt.title('Tully-Fisher Relation from Model')
plt.grid(True)

# Fit a power law
from scipy.optimize import curve_fit

def power_law(x, a, b):
    return a * x**b

popt, _ = curve_fit(power_law, total_masses/M_sun, np.array(v_max)/1000)
plt.loglog(total_masses/M_sun, power_law(total_masses/M_sun, *popt), 'r--', label=f'Fit: V ∝ M^{popt[1]:.2f}')
plt.legend()
plt.show()

print(f"Fitted power-law exponent: {popt[1]:.2f}")
print(f"Tully-Fisher relation typically has V ∝ M^0.25")

## 10. Dynamical Friction

Let's implement a simple model of dynamical friction and see how it affects the orbit of a satellite galaxy.

In [ ]:
def dynamical_friction(r, v, M_sat, rho):
    G = 6.67430e-11  # gravitational constant
    ln_lambda = 10  # Coulomb logarithm, typically 10-20
    X = v / (np.sqrt(2) * np.std(v))
    return -4 * np.pi * G**2 * M_sat * rho * ln_lambda * (erf(X) - 2*X*np.exp(-X**2)/np.sqrt(np.pi)) / v**2

def satellite_orbit(r0, v0, M_sat, t_max, dt):
    t = np.arange(0, t_max, dt)
    r = np.zeros((len(t), 2))
    v = np.zeros((len(t), 2))
    r[0] = r0
    v[0] = v0
    
    for i in range(1, len(t)):
        r_mag = np.linalg.norm(r[i-1])
        v_mag = np.linalg.norm(v[i-1])
        M_enc = enclosed_mass(np.array([r_mag]), base_galaxy_params)[0]
        rho = mass_density(r_mag, base_galaxy_params)
        
        a_grav = -G * M_enc * r[i-1] / r_mag**3
        a_fric = dynamical_friction(r_mag, v_mag, M_sat, rho) * v[i-1] / v_mag
        
        v[i] = v[i-1] + (a_grav + a_fric) * dt
        r[i] = r[i-1] + v[i] * dt
    
    return r, v, t

# Set up initial conditions
r0 = np.array([20e3*pc, 0])
v0 = np.array([0, 100e3])  # m/s
M_sat = 1e9 * M_sun
t_max = 5e9 * 365.25 * 24 * 3600  # 5 Gyr in seconds
dt = 1e6 * 365.25 * 24 * 3600  # 1 Myr in seconds

r, v, t = satellite_orbit(r0, v0, M_sat, t_max, dt)

plt.figure(figsize=(12, 12))
plt.plot(r[:, 0]/pc, r[:, 1]/pc)
plt.xlabel('x (pc)')
plt.ylabel('y (pc)')
plt.title('Satellite Galaxy Orbit with Dynamical Friction')
plt.axis('equal')
plt.grid(True)
plt.show()

plt.figure(figsize=(12, 6))
plt.plot(t / (1e9 * 365.25 * 24 * 3600), np.linalg.norm(r, axis=1) / 1e3 / pc)
plt.xlabel('Time (Gyr)')
plt.ylabel('Orbital Radius (kpc)')
plt.title('Satellite Galaxy Orbital Decay')
plt.grid(True)
plt.show()

## Conclusion

In this notebook, we've explored various aspects of our galactic dynamics model:

1. Mass distribution and enclosed mass
2. Gas properties including density, pressure, and turbulence
3. Magnetic fields and cosmic ray pressure
4. Rotation curves and their dependence on different parameters
5. Comparison with mock observational data
6. Dark matter halo profile and comparison with common models
7. Mass-to-light ratio as a function of radius
8. Tully-Fisher relation arising from our model
9. Dynamical friction effects on satellite galaxy orbits

This model provides a comprehensive framework for studying galaxy dynamics, incorporating various physical processes. However, it's important to note that this is still a simplified model, and real galaxies can exhibit much more complex behavior due to factors not included here, such as non-axisymmetric features, external perturbations, and detailed star formation and feedback processes.

Future work could focus on:
- Incorporating more detailed models of star formation and feedback
- Modeling non-axisymmetric features like bars and spiral arms
- Implementing a full N-body simulation to study galaxy evolution
- Investigating galaxy interactions and mergers
- Exploring the effects of active galactic nuclei (AGN) on galaxy dynamics

These extensions would provide an even more comprehensive understanding of galactic dynamics and evolution.

## 11. Vertical Structure of the Galactic Disk

Let's explore the vertical structure of the galactic disk by implementing a simple model for the vertical gravitational potential and density distribution.

In [ ]:
def vertical_potential(z, R, galaxy_params):
    r_d, M_d, r_b, M_b, r_h, M_h = galaxy_params
    # Approximate the vertical potential as that of an infinite sheet
    sigma_d = M_d / (2 * np.pi * r_d**2) * np.exp(-R / r_d)
    return 2 * np.pi * G * sigma_d * np.abs(z)

def vertical_density(z, R, galaxy_params, T=1e4):
    # Assume isothermal sheet model
    phi = vertical_potential(z, R, galaxy_params)
    rho_0 = mass_density(R, galaxy_params)
    return rho_0 * np.exp(-phi / (k_B * T / m_p))

# Calculate vertical structure at different radii
R_values = [5e3*pc, 10e3*pc, 20e3*pc]
z = np.linspace(-1e3*pc, 1e3*pc, 1000)

plt.figure(figsize=(12, 8))
for R in R_values:
    rho = vertical_density(z, R, base_galaxy_params)
    plt.semilogy(z/pc, rho / rho.max(), label=f'R = {R/1e3/pc:.0f} kpc')

plt.xlabel('Height above galactic plane (pc)')
plt.ylabel('Normalized Density')
plt.title('Vertical Density Distribution of Galactic Disk')
plt.legend()
plt.grid(True)
plt.show()

# Calculate scale height at different radii
R_range = np.logspace(np.log10(1e3*pc), np.log10(50e3*pc), 100)
scale_heights = []

for R in R_range:
    rho = vertical_density(z, R, base_galaxy_params)
    scale_height = np.abs(z[np.argmin(np.abs(rho - rho.max()/np.e))])
    scale_heights.append(scale_height)

plt.figure(figsize=(12, 8))
plt.loglog(R_range/pc, np.array(scale_heights)/pc)
plt.xlabel('Radius (pc)')
plt.ylabel('Scale Height (pc)')
plt.title('Disk Scale Height vs. Radius')
plt.grid(True)
plt.show()

## 12. Galaxy Scaling Relations

Let's explore how various galaxy properties scale with each other in our model, and compare these to observed scaling relations.

In [ ]:
# Generate mock observational data
r_obs = np.logspace(np.log10(1e3*pc), np.log10(50e3*pc), 20)
v_obs = solve_velocity(r_obs, base_galaxy_params, base_gas_params, method='rk4')
v_obs += np.random.normal(0, 5e3, v_obs.shape)  # Add some noise

# Calculate model prediction
v_model = solve_velocity(r, base_galaxy_params, base_gas_params, method='rk4')

plt.figure(figsize=(12, 8))
plt.semilogx(r/pc, v_model/1000, label='Model')
plt.errorbar(r_obs/pc, v_obs/1000, yerr=5, fmt='o', label='Mock Observations')
plt.xlabel('Radius (pc)')
plt.ylabel('Rotation Velocity (km/s)')
plt.title('Galaxy Rotation Curve: Model vs. Mock Observations')
plt.legend()
plt.grid(True)
plt.show()

# Calculate residuals
v_model_interp = np.interp(r_obs, r, v_model)
residuals = (v_obs - v_model_interp) / 1000  # in km/s

plt.figure(figsize=(12, 6))
plt.semilogx(r_obs/pc, residuals, 'o')
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Radius (pc)')
plt.ylabel('Residual Velocity (km/s)')
plt.title('Residuals: Observations - Model')
plt.grid(True)
plt.show()

print(f"RMS of residuals: {np.sqrt(np.mean(residuals**2)):.2f} km/s")


In [ ]:
def galaxy_properties(scale_factor):
    galaxy_params = scale_galaxy(base_galaxy_params, scale_factor)
    r_eff = galaxy_params[0]  # Assume effective radius is disk scale length
    M_total = sum(galaxy_params[1::2])
    v_max = np.max(solve_velocity(r, galaxy_params, base_gas_params, method='rk4'))
    sigma = v_max / np.sqrt(2)  # Approximate velocity dispersion
    return r_eff, M_total, v_max, sigma

scale_factors = np.logspace(-1, 1, 50)
properties = np.array([galaxy_properties(s) for s in scale_factors])

r_eff, M_total, v_max, sigma = properties.T

# Plot size-mass relation
plt.figure(figsize=(12, 8))
plt.loglog(M_total/M_sun, r_eff/pc, 'o')
plt.xlabel('Total Mass (M_sun)')
plt.ylabel('Effective Radius (pc)')
plt.title('Size-Mass Relation')
plt.grid(True)
plt.show()

# Plot Faber-Jackson relation
plt.figure(figsize=(12, 8))
plt.loglog(M_total/M_sun, sigma/1000, 'o')
plt.xlabel('Total Mass (M_sun)')
plt.ylabel('Velocity Dispersion (km/s)')
plt.title('Faber-Jackson Relation')
plt.grid(True)
plt.show()

# Plot fundamental plane
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(np.log10(r_eff/pc), np.log10(sigma/1000), np.log10(M_total/M_sun))
ax.set_xlabel('log R_eff (pc)')
ax.set_ylabel('log σ (km/s)')
ax.set_zlabel('log M_total (M_sun)')
ax.set_title('Fundamental Plane')
plt.show()

# Fit fundamental plane
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

X = np.column_stack((np.log10(r_eff/pc), np.log10(sigma/1000), np.log10(M_total/M_sun)))
X_scaled = StandardScaler().fit_transform(X)
pca = PCA(n_components=3)
pca.fit(X_scaled)

print("Fundamental Plane coefficients:")
print(f"log R_eff: {pca.components_[2,0]:.2f}")
print(f"log σ: {pca.components_[2,1]:.2f}")
print(f"log M_total: {pca.components_[2,2]:.2f}")
print(f"Variance explained: {pca.explained_variance_ratio_[2]*100:.2f}%")

## 13. Gas Depletion and Star Formation

Let's implement a simple model for gas depletion due to star formation and see how it affects the galaxy's properties over time.

In [ ]:
def star_formation_rate(r, galaxy_params, gas_params):
    r_g, M_g, T, v_turb = gas_params[:4]
    rho_gas = gas_density(r, r_g, M_g)
    # Use Kennicutt-Schmidt law: SFR ∝ ρ_gas^1.5
    return 0.01 * (rho_gas / (1e6 * m_p))**1.5  # in M_sun/yr/m^3

def evolve_galaxy(galaxy_params, gas_params, t_max, dt):
    t = np.arange(0, t_max, dt)
    M_stellar = [sum(galaxy_params[1::2]) - gas_params[1]]  # Initial stellar mass
    M_gas = [gas_params[1]]  # Initial gas mass
    SFR = []
    
    for _ in t[1:]:
        r = np.logspace(np.log10(100*pc), np.log10(100e3*pc), 1000)
        sfr = star_formation_rate(r, galaxy_params, gas_params)
        dM = np.sum(sfr * 4 * np.pi * r**2 * np.diff(r)[0]) * dt * 365.25 * 24 * 3600  # Mass converted to stars
        
        M_stellar.append(M_stellar[-1] + dM)
        M_gas.append(M_gas[-1] - dM)
        SFR.append(np.sum(sfr * 4 * np.pi * r**2 * np.diff(r)[0]))  # Total SFR in M_sun/yr
        
        # Update gas parameters
        gas_params = (gas_params[0], M_gas[-1]) + gas_params[2:]
    
    return t, np.array(M_stellar), np.array(M_gas), np.array(SFR)

t_max = 10e9 * 365.25 * 24 * 3600  # 10 Gyr in seconds
dt = 1e8 * 365.25 * 24 * 3600  # 100 Myr in seconds

t, M_stellar, M_gas, SFR = evolve_galaxy(base_galaxy_params, base_gas_params, t_max, dt)

plt.figure(figsize=(12, 8))
plt.plot(t / (1e9 * 365.25 * 24 * 3600), M_stellar / M_sun, label='Stellar Mass')
plt.plot(t / (1e9 * 365.25 * 24 * 3600), M_gas / M_sun, label='Gas Mass')
plt.xlabel('Time (Gyr)')
plt.ylabel('Mass (M_sun)')
plt.title('Galaxy Mass Evolution')
plt.legend()
plt.grid(True)
plt.show()

plt.figure(figsize=(12, 8))
plt.semilogy(t[1:] / (1e9 * 365.25 * 24 * 3600), SFR)
plt.xlabel('Time (Gyr)')
plt.ylabel('Star Formation Rate (M_sun/yr)')
plt.title('Star Formation Rate Evolution')
plt.grid(True)
plt.show()

## 14. Galaxy Merger Simulation

Let's implement a simple galaxy merger simulation using our model.

In [ ]:
def merge_galaxies(galaxy1_params, galaxy2_params, r1_initial, v1_initial, r2_initial, v2_initial, t_max, dt):
    t = np.arange(0, t_max, dt)
    r1 = np.zeros((len(t), 3))
    r2 = np.zeros((len(t), 3))
    v1 = np.zeros((len(t), 3))
    v2 = np.zeros((len(t), 3))
    
    r1[0], r2[0] = r1_initial, r2_initial
    v1[0], v2[0] = v1_initial, v2_initial
    
    M1 = sum(galaxy1_params[1::2])
    M2 = sum(galaxy2_params[1::2])
    
    for i in range(1, len(t)):
        r = r2[i-1] - r1[i-1]
        r_mag = np.linalg.norm(r)
        
        # Gravitational acceleration
        a1 = G * M2 * r / r_mag**3
        a2 = -G * M1 * r / r_mag**3
        
        # Dynamical friction
        rho1 = mass_density(r_mag, galaxy1_params)
        rho2 = mass_density(r_mag, galaxy2_params)
        v_rel = v2[i-1] - v1[i-1]
        v_rel_mag = np.linalg.norm(v_rel)
        a1_fric = dynamical_friction(r_mag, v_rel_mag, M2, rho1) * v_rel / v_rel_mag
        a2_fric = dynamical_friction(r_mag, v_rel_mag, M1, rho2) * -v_rel / v_rel_mag
        
        # Update velocities and positions
        v1[i] = v1[i-1] + (a1 + a1_fric) * dt
        v2[i] = v2[i-1] + (a2 + a2_fric) * dt
        r1[i] = r1[i-1] + v1[i] * dt
        r2[i] = r2[i-1] + v2[i] * dt
        
        # Check for merger completion
        if r_mag < (galaxy1_params[0] + galaxy2_params[0]):
            return r1[:i+1], r2[:i+1], v1[:i+1], v2[:i+1], t[:i+1]
    
    return r1, r2, v1, v2, t

# Set up initial conditions for merger
galaxy1_params = base_galaxy_params
galaxy2_params = scale_galaxy(base_galaxy_params, 0.5)  # Smaller galaxy

r1_initial = np.array([0, 0, 0])
r2_initial = np.array([100e3*pc, 0, 0])
v1_initial = np.array([0, 0, 0])
v2_initial = np.array([0, 100e3, 0])  # Tangential velocity

t_max = 5e9 * 365.25 * 24 * 3600  # 5 Gyr in seconds
dt = 1e6 * 365.25 * 24 * 3600  # 1 Myr in seconds

r1, r2, v1, v2, t = merge_galaxies(galaxy1_params, galaxy2_params, r1_initial, v1_initial, r2_initial, v2_initial, t_max, dt)

# Plot merger trajectory
plt.figure(figsize=(12, 12))
plt.plot(r1[:, 0]/pc, r1[:, 1]/pc, label='Galaxy 1')
plt.plot(r2[:, 0]/pc, r2[:, 1]/pc, label='Galaxy 2')
plt.xlabel('x (pc)')
plt.ylabel('y (pc)')
plt.title('Galaxy Merger Trajectory')
plt.legend()
plt.axis('equal')
plt.grid(True)
plt.show()

# Plot separation over time
separation = np.linalg.norm(r2 - r1, axis=1)
plt.figure(figsize=(12, 8))
plt.plot(t / (1e9 * 365.25 * 24 * 3600), separation / 1e3 / pc)
plt.xlabel('Time (Gyr)')
plt.ylabel('Separation (kpc)')
plt.title('Galaxy Separation During Merger')
plt.grid(True)
plt.show()

# Animate merger (you may need to install matplotlib animation if not already installed)
from matplotlib.animation import FuncAnimation

fig, ax = plt.subplots(figsize=(10, 10))
ax.set_xlim(min(r1[:, 0].min(), r2[:, 0].min()) / pc, max(r1[:, 0].max(), r2[:, 0].max()) / pc)
ax.set_ylim(min(r1[:, 1].min(), r2[:, 1].min()) / pc, max(r1[:, 1].max(), r2[:, 1].max()) / pc)
ax.set_xlabel('x (pc)')
ax.set_ylabel('y (pc)')
ax.set_title('Galaxy Merger Animation')

line1, = ax.plot([], [], 'bo', markersize=10, label='Galaxy 1')
line2, = ax.plot([], [], 'ro', markersize=8, label='Galaxy 2')
time_text = ax.text(0.02, 0.95, '', transform=ax.transAxes)

def init():
    line1.set_data([], [])
    line2.set_data([], [])
    time_text.set_text('')
    return line1, line2, time_text

def animate(i):
    line1.set_data(r1[i, 0] / pc, r1[i, 1] / pc)
    line2.set_data(r2[i, 0] / pc, r2[i, 1] / pc)
    time_text.set_text(f'Time: {t[i] / (1e9 * 365.25 * 24 * 3600):.2f} Gyr')
    return line1, line2, time_text

anim = FuncAnimation(fig, animate, init_func=init, frames=len(t), interval=50, blit=True)
plt.legend()
plt.grid(True)
plt.show()
# If you want to save the animation (optional):
# anim.save('galaxy_merger.gif', writer='pillow', fps=30)

## 15. Effect of Active Galactic Nucleus (AGN) Feedback

Let's implement a simple model of AGN feedback and see how it affects the galaxy's gas distribution and star formation rate.

In [ ]:
def agn_feedback(r, L_agn, v_out=0.1*c):
    # Simple model: AGN imparts momentum to gas
    P_agn = L_agn / c
    return P_agn / (4 * np.pi * r**2 * v_out)

def evolve_galaxy_with_agn(galaxy_params, gas_params, L_agn, t_max, dt):
    t = np.arange(0, t_max, dt)
    M_stellar = [sum(galaxy_params[1::2]) - gas_params[1]]  # Initial stellar mass
    M_gas = [gas_params[1]]  # Initial gas mass
    SFR = []
    
    for _ in t[1:]:
        r = np.logspace(np.log10(100*pc), np.log10(100e3*pc), 1000)
        sfr = star_formation_rate(r, galaxy_params, gas_params)
        
        # Apply AGN feedback
        agn_force = agn_feedback(r, L_agn)
        rho_gas = gas_density(r, *gas_params[:2])
        v_escape = np.sqrt(2 * G * enclosed_mass(r, galaxy_params) / r)
        mask_outflow = agn_force / rho_gas > v_escape
        sfr[mask_outflow] = 0  # No star formation in outflow regions
        
        dM_sf = np.sum(sfr * 4 * np.pi * r**2 * np.diff(r)[0]) * dt * 365.25 * 24 * 3600  # Mass converted to stars
        dM_out = np.sum(rho_gas[mask_outflow] * 4 * np.pi * r[mask_outflow]**2 * np.diff(r)[0])  # Mass in outflow
        
        M_stellar.append(M_stellar[-1] + dM_sf)
        M_gas.append(M_gas[-1] - dM_sf - dM_out)
        SFR.append(np.sum(sfr * 4 * np.pi * r**2 * np.diff(r)[0]))  # Total SFR in M_sun/yr
        
        # Update gas parameters
        gas_params = (gas_params[0], M_gas[-1]) + gas_params[2:]
    
    return t, np.array(M_stellar), np.array(M_gas), np.array(SFR)

# Run simulations with and without AGN feedback
t_max = 5e9 * 365.25 * 24 * 3600  # 5 Gyr in seconds
dt = 1e7 * 365.25 * 24 * 3600  # 10 Myr in seconds

t_no_agn, M_stellar_no_agn, M_gas_no_agn, SFR_no_agn = evolve_galaxy(base_galaxy_params, base_gas_params, t_max, dt)

L_agn = 1e38  # AGN luminosity in W (about 10^11 L_sun)
t_agn, M_stellar_agn, M_gas_agn, SFR_agn = evolve_galaxy_with_agn(base_galaxy_params, base_gas_params, L_agn, t_max, dt)

# Plot results
plt.figure(figsize=(12, 8))
plt.plot(t_no_agn / (1e9 * 365.25 * 24 * 3600), M_stellar_no_agn / M_sun, label='Stellar Mass (No AGN)')
plt.plot(t_no_agn / (1e9 * 365.25 * 24 * 3600), M_gas_no_agn / M_sun, label='Gas Mass (No AGN)')
plt.plot(t_agn / (1e9 * 365.25 * 24 * 3600), M_stellar_agn / M_sun, '--', label='Stellar Mass (With AGN)')
plt.plot(t_agn / (1e9 * 365.25 * 24 * 3600), M_gas_agn / M_sun, '--', label='Gas Mass (With AGN)')
plt.xlabel('Time (Gyr)')
plt.ylabel('Mass (M_sun)')
plt.title('Galaxy Mass Evolution with and without AGN Feedback')
plt.legend()
plt.grid(True)
plt.show()

plt.figure(figsize=(12, 8))
plt.semilogy(t_no_agn[1:] / (1e9 * 365.25 * 24 * 3600), SFR_no_agn, label='No AGN')
plt.semilogy(t_agn[1:] / (1e9 * 365.25 * 24 * 3600), SFR_agn, '--', label='With AGN')
plt.xlabel('Time (Gyr)')
plt.ylabel('Star Formation Rate (M_sun/yr)')
plt.title('Star Formation Rate Evolution with and without AGN Feedback')
plt.legend()
plt.grid(True)
plt.show()

# Calculate and plot the gas density profile at different times
def gas_density_profile(r, M_gas, r_g):
    return M_gas / (2 * np.pi * r_g**2) * np.exp(-r / r_g)

r = np.logspace(np.log10(100*pc), np.log10(100e3*pc), 1000)
times = [0, 1, 2, 5]  # Gyr
plt.figure(figsize=(12, 8))

for t in times:
    idx = int(t * 1e9 / (dt / (365.25 * 24 * 3600)))
    rho_no_agn = gas_density_profile(r, M_gas_no_agn[idx], base_gas_params[0])
    rho_agn = gas_density_profile(r, M_gas_agn[idx], base_gas_params[0])
    plt.loglog(r/pc, rho_no_agn, label=f'No AGN, t={t} Gyr')
    plt.loglog(r/pc, rho_agn, '--', label=f'With AGN, t={t} Gyr')

plt.xlabel('Radius (pc)')
plt.ylabel('Gas Density (kg/m³)')
plt.title('Gas Density Profile Evolution')
plt.legend()
plt.grid(True)
plt.show()

## Conclusion

In this comprehensive exploration of our galactic dynamics model, we've covered a wide range of phenomena and their effects on galaxy evolution:

1. Mass distribution and enclosed mass
2. Gas properties including density, pressure, and turbulence
3. Magnetic fields and cosmic ray pressure
4. Rotation curves and their dependence on different parameters
5. Comparison with mock observational data
6. Dark matter halo profile and comparison with common models
7. Mass-to-light ratio as a function of radius
8. Tully-Fisher relation arising from our model
9. Dynamical friction effects on satellite galaxy orbits
10. Vertical structure of the galactic disk
11. Galaxy scaling relations (size-mass, Faber-Jackson, fundamental plane)
12. Gas depletion and star formation over time
13. Galaxy merger simulations
14. Effects of AGN feedback on galaxy evolution

This model provides a comprehensive framework for studying galaxy dynamics and evolution, incorporating various physical processes. However, it's important to note that this is still a simplified model, and real galaxies can exhibit much more complex behavior due to factors not fully included here, such as detailed gas physics, complex feedback mechanisms, and environmental effects.

Future work could focus on:
- Incorporating more detailed models of star formation and feedback
- Implementing a full hydrodynamical treatment of the gas
- Including the effects of supermassive black hole growth and its coupling to galaxy evolution
- Modeling the chemical evolution of galaxies
- Investigating the effects of cosmic environment on galaxy evolution
- Implementing machine learning techniques to compare model predictions with large observational datasets

These extensions would provide an even more comprehensive understanding of galactic dynamics and evolution, bridging the gap between theoretical models and observational data.